In [14]:
import pandas as pd
import numpy as np
from repo_funciones import *

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('../Sharknado/attacks_messy.csv', encoding='utf-8',encoding_errors="ignore" )

Exploración Inicial

In [16]:
dim_original = data.shape #Registramos dimensiones iniciales del archivo para luego ir comparando
dim_original

(25723, 24)

In [17]:
#data.head() Usamos estas funciones para ir haciendonos una idea de los datos y las columnas
#data.tail() Con este nos damos cuenta de que al final del archivo hay filas enteras con valores nulos

In [18]:
#data.info() #Estimación de que columnas necesitan mas trabajo. Por gusto mío prefiero verlo a la inversa -> que columnas tienen mas valores nulos.

In [19]:
nancols(data) #Conteo de valores nulos por col, ascending False.

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
Date                      19421
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Case Number               17021
dtype: int64

In [20]:
nancolsper(data) #conteo de valores nulos como % del total de la columna. Nos da una mejor estimación 

Unnamed: 22               100.00
Unnamed: 23                99.99
Time                       88.54
Species                    86.53
Age                        86.51
Sex                        77.70
Activity                   77.62
Location                   77.60
Fatal (Y/N)                77.60
Area                       77.27
Name                       76.32
Country                    75.69
Injury                     75.61
Investigator or Source     75.57
Type                       75.52
Year                       75.51
href formula               75.50
Date                       75.50
pdf                        75.50
href                       75.50
Case Number.1              75.50
Case Number.2              75.50
original order             75.47
Case Number                66.17
dtype: float64

Limpieza duplicados y columnas vacias

In [21]:
data= data.drop_duplicates()

In [22]:
data.duplicated().unique()

array([False])

In [24]:
data=data.drop(nanrows(data)[nanrows(data)> 19]) #Si en una fila 90% de los valores son nulos, se descartan

In [26]:
data = data.reset_index(drop=True)

In [27]:
data = data.drop(nanrows(data)[nanrows(data) >= 15]) #Si en una fila 60% de los valores son nulos, se descartan.

In [28]:
dim_original, data.shape

((25723, 24), (6306, 24))

In [29]:
data = data.reset_index(drop=True)

In [30]:
nancolsper(data) #Unamed 22 y 23 estan vacías y no aportan nada. Se descartan.

Unnamed: 22               99.98
Unnamed: 23               99.97
Time                      53.30
Species                   45.15
Age                       45.04
Sex                        9.12
Activity                   8.79
Location                   8.72
Fatal (Y/N)                8.69
Area                       7.37
Name                       3.49
Country                    0.95
Injury                     0.60
Investigator or Source     0.43
Type                       0.22
Year                       0.19
href formula               0.17
Date                       0.16
pdf                        0.16
href                       0.16
Case Number.1              0.16
Case Number.2              0.16
original order             0.05
Case Number                0.03
dtype: float64

In [32]:
data.drop(["Unnamed: 22", "Unnamed: 23"], axis=1, inplace=True)  #Reminder, axis 0 son filas y axis columnas

In [33]:
nancolsper(data)

Time                      53.30
Species                   45.15
Age                       45.04
Sex                        9.12
Activity                   8.79
Location                   8.72
Fatal (Y/N)                8.69
Area                       7.37
Name                       3.49
Country                    0.95
Injury                     0.60
Investigator or Source     0.43
Type                       0.22
Year                       0.19
href formula               0.17
Date                       0.16
Case Number.2              0.16
pdf                        0.16
href                       0.16
Case Number.1              0.16
original order             0.05
Case Number                0.03
dtype: float64

Limpieza Columna "Time"

In [35]:
#data["Time"].unique() #Vemos hay muchas inconsistencias en los valores, hay que homogenizar

In [38]:
#unique_cols(data) #364 valores unicos 
#get_unique_dtypes(data, "Time") #con strings y con floats. Se intuye floats seran los valores nulos.

In [39]:
data["Time"]=data["Time"].fillna("unknown")

In [40]:
data["Time"] = data["Time"].str.replace('h',':')